---
title: Comparing the predicted reference epigenome to Enformer runs centered at the TSS and ground truth
author: Sabrina Mi
date: 8/28/2023
---

We are trying to debug low correlation values between predicted reference CAGE values and reference observed gene expression. We'll start by selecting 10 genes.

In [7]:
import pandas as pd
annot_df = pd.read_csv("/Users/sabrinami/Github/deep-learning-in-genomics/posts/2023-08-18-gene-annotation-query-example-hg38/protein_coding_TSS.tsv", sep="\t")
genes = annot_df.sample(n=10)

In [8]:
intervals = [f"chr{row['chromosome_name']}_{row['transcription_start_site']}_{row['transcription_start_site']}" 
            for _, row in genes.iterrows()]
print(intervals)
with open("metadata/intervals.txt", "w") as file:
    file.write("\n".join(intervals))

['chrX_73079512_73079512', 'chr12_7018476_7018476', 'chr3_114294028_114294028', 'chr19_36528271_36528271', 'chr13_52642431_52642431', 'chr7_135148072_135148072', 'chr2_241102287_241102287', 'chr17_82371765_82371765', 'chr2_55050451_55050451', 'chr1_235649784_235649784']
